### Simple prompt

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt = PromptTemplate(
    template="Generate 5 interresting facts about {topic}",
    input_variables=["topic"],
)

model = ChatOpenAI()

parser = StrOutputParser()

chain = prompt | model | parser
result = chain.invoke({"topic": "Python programming"})
print(result)


1. Python was created by Guido van Rossum and first released in 1991. It is named after the British comedy group Monty Python.

2. Python is a high-level, interpreted programming language known for its simplicity and readability. It uses indentation to define code blocks, making it easier to read and understand compared to other languages.

3. Python is used in a wide range of applications, including web development, data analysis, artificial intelligence, machine learning, and automation. It is one of the most popular languages for data science and is widely used by companies like Google, Facebook, and NASA.

4. Python has a large and active community of developers who contribute to its open-source ecosystem. There are many libraries and frameworks available for Python, making it easy to extend and customize for specific applications.

5. Python is cross-platform and runs on all major operating systems, including Windows, macOS, and Linux. It is free to use and distribute, making it a

### Sequential Chain

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI()

template1 = PromptTemplate(
    template= "Generate a detailed report about {topic}",
    input_variables=["topic"],
)

template2 = PromptTemplate(
    template = "Generate 5 pointer summmary from the following text \n {text}",
    input_variables=["text"],
)

parser = StrOutputParser()
chain = template1 | model | parser | template2 | model | parser
result = chain.invoke({"topic": "AI Agents"})
print(result)

1. AI agents are software programs designed to perform tasks autonomously based on predefined rules, algorithms, or machine learning models in various fields such as healthcare, finance, manufacturing, and transportation.
2. They can be categorized into reactive, deliberative, and hybrid agents based on their decision-making processes and capabilities.
3. AI agents can operate in single-agent systems independently or as part of multi-agent systems that collaborate with each other to achieve common goals.
4. They can be implemented using various technologies such as machine learning, expert systems, reinforcement learning, and natural language processing.
5. The applications of AI agents are diverse, including controlling autonomous vehicles, robots in manufacturing, assisting in healthcare, analyzing financial data, and predicting market trends.


### Parallel chain

In [9]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI()
model2 = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

prompt1 = PromptTemplate(
    template = "Generata short and simple notes from the following text \n {text}",
    input_variables=["text"],
)

prompt2 = PromptTemplate(
    template = "Generate 5 short questions answer from the following text \n {text}",
    input_variables=["text"],
)

prompt3 = PromptTemplate(
    template = "Merge the provided notes and quiz into a sinlgle document \ notes -> {notes} \ quiz -> {quiz}",
    input_variables=["notes", "quiz"],
)

parser = StrOutputParser()

parallel_chain = RunnableParallel(
    {
        "notes": prompt1 | model | parser,
        "quiz": prompt2 | model2 | parser,
    }
)

chain = parallel_chain | prompt3 | model | parser
result = chain.invoke({"text": "AI Agents are software programs that can perform tasks autonomously. They can learn from their environment and adapt to new situations. AI agents are used in various fields, including robotics, finance, and healthcare."})
print(result)

**Notes:**
- AI agents are software programs that work autonomously
- They can learn from their environment and adapt to new situations
- Used in fields such as robotics, finance, and healthcare

**Quiz:**

**Questions & Answers:**

1.  **Question:** What are AI Agents?
    **Answer:** Software programs that can perform tasks autonomously.

2.  **Question:** What can AI Agents learn from?
    **Answer:** Their environment.

3.  **Question:** What can AI Agents adapt to?
    **Answer:** New situations.

4.  **Question:** Name one field where AI Agents are used.
    **Answer:** Robotics (or Finance, or Healthcare).

5.  **Question:** What is a key characteristic of AI Agents?
    **Answer:** Autonomy (or the ability to learn, or the ability to adapt).


### Conditional Chain

In [35]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI()

parser = StrOutputParser()

# class SentimentModel(BaseModel):
#     sentiment: str = Field(description="The sentiment of the text")

#     class Config:
#         schema_extra = {
#             "examples": [
#                 {"sentiment": "positive"},
#                 {"sentiment": "negative"},
#                 {"sentiment": "neutral"},
#             ]
#         }

# Uncomment the following lines to use PydanticOutputParser
class FeedbackModel(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="The sentiment of the text")

parser1 = PydanticOutputParser(pydantic_object=FeedbackModel)

prompt1 = PromptTemplate(
    template="Classify the sentiment of the following text as positive, negative, or neutral: {text} \n {format_instructions}",
    input_variables=["text"],
    partial_variables={"format_instructions": parser1.get_format_instructions()},
)

classification_chain = prompt1 | model | parser1

# result = classification_chain.invoke({"text": "I love programming with Python!"})


from langchain.schema.runnable import RunnableLambda, RunnableBranch

# Fallback in case no sentiment matches (very unlikely, but needed for safety)
default_chain = RunnableLambda(lambda x: "Could not determine sentiment.")

prompt2 = PromptTemplate(
    template ="Write a appropriate response to this positive sentiment \n  {sentiment}",
    input_variables=["sentiment"],
)

prompt3 = PromptTemplate(
    template ="Write a appropriate response to this negative sentiment \n  {sentiment}",
    input_variables=["sentiment"],
)

prompt4 = PromptTemplate(
    template ="Write a appropriate response to this neutral  sentiment \n {sentiment}",
    input_variables=["sentiment"],
)

condition1 = RunnableLambda(lambda x: x.sentiment == "positive")
condition2 = RunnableLambda(lambda x: x.sentiment == "negative")
condition3 = RunnableLambda(lambda x: x.sentiment == "neutral")


chain1 = prompt2 | model | parser
chain2 = prompt3 | model | parser
chain3 = prompt4 | model | parser

default_chain = RunnableLambda(lambda x: "Could not determine sentiment.")

branch_chain = RunnableBranch(
    (condition1, chain1),
    (condition2, chain2),
    (condition3, chain3),
    default_chain,
)

# Combine the classification and branching chains
final_chain = classification_chain | branch_chain
result = final_chain.invoke({"text": "Your product is amazing! I love it."})
print(result)

Thank you for the kind words! It means a lot to me.
